## Import Libraries

In [46]:
import requests
import pandas as pd
import time
from datetime import datetime

In [47]:
# List of Selected Tickers we want to analyse 
tickers = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'TSLA']

In [93]:
# store stockdata here 
stock_data = []

### API Calls for Multiple Tickers
#### Storing data to Stock_Data
Transforming open, close, high, low from str to floats,
Date to date object 
volume from str to int

In [ ]:
# outputsize defaults to compact if you want 20 years of data you can change to "= full" 
# monthly data defaults to full - they returned me over 500 data points for 1 ticker 

api_key = "9JPRF2OKFQM0FT0U"
base_url_template = 'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={symbol}&apikey={api_key}&outputsize=compact'

for tick in tickers:
    url = base_url_template.format(symbol=tick, api_key=api_key)
    try:
        r = requests.get(url)
        r.raise_for_status()  
        data = r.json()

        time_series_key = 'Time Series (Daily)' # Depends on what key is returned by the specified API function 
        if time_series_key in data:
            for date_str, values in data[time_series_key].items():
                row = {
                    'Symbol': tick,
                    'Date': date_str,  
                    'Open': float(values['1. open']),
                    'High': float(values['2. high']),
                    'Low': float(values['3. low']),
                    'Close': float(values['4. close']),
                    'Volume': int(values['5. volume'])
                }
                stock_data.append(row)
            print(f"--- Successfully collected and formatted data for {tick} ---")
            
        elif "Error Message" in data:
            print(f"API Error for {tick}: {data['Error Message']}")
        else:
            print(f"Unexpected data format for {tick}.") 
        if "Error Message" in data:
            print(f"Error for {tick}: {data['Error Message']}")

        else:
            print(f"--- Data for {tick} ---")
            print(data)
    except requests.exceptions.RequestException as e:
        print(f"An error occurred while fetching data for {tick}: {e}")
    time.sleep(20)    



In [ ]:
# convert to DataFrame  
stock_df = pd.DataFrame(stock_data)

# convert date from str to date time
stock_df['Date'] = pd.to_datetime(stock_df['Date'])

## Check the number of entries per ticker 

In [ ]:
print(stock_df['Symbol'].value_counts())
print(stock_df.head(20))

## Convert to CSV

In [83]:
stock_df.to_csv('stock_data.csv', index=False)